In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
# cwd = os.getcwd()
# print("Current working directory is:",cwd)
!pwd

/content


In [0]:
# os.chdir("/content/drive/My Drive/CA-684-ML/dev-set")
# os.chdir("/content/drive/Shared with me/CA684_Assignment/Dev-set")
os.chdir("/content/drive/My Drive/CA684_Assignment/Dev-set")

In [0]:
!pwd

/content/drive/.shortcut-targets-by-id/1UWIdExqb5P94RLqkaiODcwFC2v7g3lB2/CA684_Assignment/Dev-set


In [0]:
!ls

'Aesthetic Features'   HMP				     ORB
 C3D		       HOG				     README.txt
 Captions	       InceptionV3			     Sources
 ColorHistogram        LBP
 Ground-truth	       MemorabilityDataset2018_license.txt


In [0]:
ground_truth = pd.read_csv('Ground-truth/ground-truth.csv', header=0)

In [0]:
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn import linear_model
from sklearn.linear_model import BayesianRidge
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from scipy.stats import spearmanr
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import scipy as sp
from sklearn.model_selection import KFold
from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
ground_truth.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


In [0]:
# !pwd
captions_dataset = pd.read_csv('Captions/dev-set_video-captions.txt', sep = '\t', header=None, names=['video', 'caption'])
# captions_dataset = pd.read_csv('dev-set_video-captions.txt', sep = '\t', header=None, names=['video', 'caption'] )

In [0]:
captions_dataset.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [0]:
ground_truth.shape

(6000, 5)

In [0]:
captions_dataset.shape

(6000, 2)

In [0]:
merged_1 = ground_truth.merge(captions_dataset)

In [0]:
merged_1.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations,caption
0,video3.webm,0.924,34,0.846,13,blonde-woman-is-massaged-tilt-down
1,video4.webm,0.923,33,0.667,12,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,0.863,33,0.700,10,khr-gangsters
3,video8.webm,0.922,33,0.818,11,medical-helicopter-hovers-at-airport
4,video10.webm,0.950,34,0.900,10,couple-relaxing-on-picnic-crane-shot


In [0]:
def read_c3d_from_file(dataframe, filename):
  dataframe_1 = pd.read_csv(filename, header=None, sep=' ')
  dataframe_1['video'] = filename[filename.find('video'):filename.find('.txt')] + '.webm'
  dataframe = dataframe.append(dataframe_1, ignore_index=True)
  return dataframe

In [0]:
def reading_file_names(path):
  files = []
  file_number=0
  # r=root, d=directories, f = files
  for r, d, f in os.walk(path):
      for file in f:
          if '.txt' in file:
              files.append(os.path.join(r, file))
              file_number = file_number + 1
  print("total files read = " + str(file_number))
  return files

In [33]:
dataframe_aesthetic.head()

NameError: ignored

In [34]:
merged_dataset = merged_1.merge(dataframe)

NameError: ignored

In [0]:
os.chdir("C3D")

In [36]:
!pwd

/content/drive/.shortcut-targets-by-id/1UWIdExqb5P94RLqkaiODcwFC2v7g3lB2/CA684_Assignment/Dev-set


In [0]:
path="/content/drive/My Drive/CA684_Assignment/Dev-set/C3D"
files = reading_file_names(path)
dataframe_C3D = pd.DataFrame()
for filename in files:
  dataframe_C3D = read_c3d_from_file(dataframe_C3D, filename)

total files read = 6000


In [0]:
dataframe_C3D.head()